In [1]:
%load_ext autoreload
%autoreload 2

from hydroml.training.train import train
from hydroml.config.config import Config, load_config
import hydroml.training.finetune as ft
from hydroml.models import get_model_from_path
from hydroml.data.camels_aus_ds import get_dataset
from hydroml.prediction.prediction import process_and_convert_dataloader_to_xarray
from hydroml.models.get_model_from_path import get_model_from_path
from hydroml.evaluation.metrics import get_metrics
from hydroml.training.trainer import get_trainer
from pathlib import Path
def plot(ds):
    ds.prediction.plot()
    ds.y.plot()
    import matplotlib.pyplot as plt
    plt.legend(['prediction', 'y'])
    plt.ylim(0, 5)
    plt.show()



In [2]:
with open('P://work//sho108//hydroml//workflows//basins.txt', 'r') as f:
    basins = f.read().splitlines()

catchment_ids = basins#'410730'
# '410730'
config = Config(cal={'periods' : [['1985-01-01', '2014-01-01']], 'catchment_ids':catchment_ids}, #, 'weights': [1, 1, 1, 1, 1, 1, 1, 1, 1]
                val={'periods' : [['1985-01-01', '2014-01-01']], 'catchment_ids':catchment_ids[0:1]}, #,
                max_epochs=15,
                lstm_hidden_size=256,
                batch_size=256,
                initial_forget_bias=0,
                parent_path = 'P://work//sho108//hydroml//results_2',
                lr=1e-3,
                dataloader_nworkers=9,
                dataloader_persistent_workers=True                
                )




In [6]:
current_path_pre, version_pre = train(config)

Transforming data: loading transform parameters from P:\work\sho108\hydroml\results_2\default\params.yaml
valid data points per catchment {0: 9853, 1: 7590, 2: 8049, 3: 9642, 4: 8473, 5: 7756, 6: 6931, 7: 8093, 8: 8417, 9: 9326, 10: 9841, 11: 8445, 12: 9364, 13: 10145, 14: 10533, 15: 8518, 16: 10473, 17: 10194, 18: 9946, 19: 10311, 20: 10124, 21: 10533, 22: 10473, 23: 10533, 24: 10473, 25: 10473, 26: 10533, 27: 10342, 28: 10437, 29: 10419, 30: 9383, 31: 10220, 32: 10432, 33: 10533, 34: 10296, 35: 10379, 36: 10339, 37: 9907, 38: 10109, 39: 10467, 40: 10466, 41: 10467, 42: 10264, 43: 10439, 44: 10533, 45: 10007, 46: 10533, 47: 10533, 48: 10533, 49: 10462, 50: 10467, 51: 10446, 52: 9784, 53: 10462, 54: 10533, 55: 10473, 56: 10466, 57: 10451, 58: 10309, 59: 10533, 60: 10284, 61: 10272, 62: 10533, 63: 9610, 64: 9407, 65: 10438, 66: 10409, 67: 10011, 68: 10533, 69: 10441, 70: 10066, 71: 10341, 72: 10302, 73: 10095, 74: 8769, 75: 10174, 76: 10470, 77: 10289, 78: 10450, 79: 10303, 80: 10264, 8

GPU available: False, used: False                                       
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Transforming data: loading transform parameters from P:\work\sho108\hydroml\results_2\default\params.yaml


c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory \\fs1-cbr.nexus.csiro.au\{ev-ca-macq}\work\sho108\hydroml\results_2\default\241210215144_6580 exists and is not empty.

  | Name              | Type       | Params | Mode 
---------------------------------------------------------
0 | static_embedding  | Linear     | 15     | train
1 | dynamic_embedding | Linear     | 5      | train
2 | lstm              | LSTM       | 266 K  | train
3 | dropout           | Identity   | 0      | train
4 | head              | Sequential | 2.6 K  | train
---------------------------------------------------------
268 K     Trainable params
0         Non-trainable params
268 K     Total params
1.075     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\pytorch_lightning\utilities\data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 102. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:384: `ModelCheckpoint(monitor='val_loss')` could not find the monitored key in the returned metrics: ['lr-Adam', 'train_loss', 'epoch', 'step']. HINT: Did you call `log('val_loss', value)` in the `LightningModule`?


Validation: |          | 0/? [00:00<?, ?it/s]

c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\pytorch_lightning\utilities\data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 125. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


: 

In [4]:

print('first pass:')
dataset_cal_train = get_dataset(config, 'cal', is_train=True)
dataloader_cal_train = dataset_cal_train.to_dataloader()
pretrained_model_path = Path('P://work//sho108//hydroml//results_2//default//241210215144_6580')
model = get_model_from_path(pretrained_model_path)
ds_pre = process_and_convert_dataloader_to_xarray(dataloader_cal_train, model, config)
m_pre = get_metrics(ds_pre)
m_pre.to_dataframe()


first pass:


Transforming data: loading transform parameters from P:\work\sho108\hydroml\results_2\default\params.yaml


c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\xarray\core\computation.py:818: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\xarray\core\computation.py:818: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


,,,nse,kge,rmse,bias,relative_bias,absolute_bias,nse_sqrt
catchment_id,lead_time,feature,,,,,,,
102101A,0,0,0.131709,-0.202987,7.970991,0.220550,-0.779450,0.220550,-0.082133
104001A,0,0,0.322257,-0.342095,3.557014,-0.161673,-1.161674,0.161673,-0.557289
105101A,0,0,0.054011,-0.521051,2.872990,-0.168064,-1.168064,0.168064,-1.928226
105102A,0,0,0.093461,-0.532059,3.326761,-0.179544,-1.179544,0.179544,-1.266895
105105A,0,0,0.564584,0.155063,3.049018,0.283162,-0.716838,0.283162,-0.022017
...,...,...,...,...,...,...,...,...,...
G8200045,0,0,0.068976,-0.454449,4.115690,-0.140012,-1.140012,0.140012,-1.626599
G8210010,0,0,0.003885,-0.497147,4.975430,-0.122652,-1.122652,0.122652,-2.880549
G9030124,0,0,-0.326678,-3.372349,1.189864,-3.342604,-4.342604,3.342604,-4.154618


AttributeError: 'DataArray' object has no attribute 'sort_values'

In [ ]:
results = {}
pretrained_model_path = current_path_pre/version_pre #Path('P://work//sho108//hydroml//results_2//default//241210160237_1337')
for i, catchment_id in enumerate(catchment_ids):
    print(f'{i+1}/{len(catchment_ids)}: {catchment_id}')

    print(f'second pass using finetuning module: {catchment_id}')
    if catchment_id in results:
        continue
    #reload_model
    model = get_model_from_path(pretrained_model_path, check_point='last')
    config = ft.update_config_for_per_catchment_finetune(config, cal_catchment_ids=[catchment_id], val_catchment_ids=[catchment_id], max_epochs=5)
    
    current_path, version = ft.finetune(model, config, finetune_directory='finetune', finetune_name=catchment_id)

    ft_model = get_model_from_path(current_path/version, check_point='last')

    ds = process_and_convert_dataloader_to_xarray(dataloader_cal_train, ft_model, config)
    m = get_metrics(ds)
    
    results[catchment_id] = m.to_dataframe()

    print('-'*100)





In [ ]:
import pandas as pd
import numpy as np
results_list = []
for k in results:
    results_list.append(results[k].loc[[k]])

fine_tuned_results  = pd.concat(results_list)


total_results = pd.concat([m_pre.to_dataframe()['nse'], fine_tuned_results['nse']], axis=1)
total_results.columns = ['pretrained', 'fine_tuned']

total_results.dropna(inplace=True)

sorted_pretrained = total_results['pretrained'].sort_values(ascending=False).values
sorted_fine_tuned = total_results['fine_tuned'].sort_values(ascending=False).values


# plot exceedance curve
import matplotlib.pyplot as plt
x = np.linspace(0,1,len(sorted_pretrained))
plt.plot(x, sorted_pretrained)
plt.plot(x, sorted_fine_tuned)
plt.legend(['pretrained', 'fine_tuned'])

plt.ylim(0,1)
plt.show()

diff = total_results['fine_tuned'] - total_results['pretrained']
sorted_diff = diff.sort_values(ascending=False).values
np.clip(sorted_diff, -1, 1, out=sorted_diff)

# count how many catchments have positive improvement
print(f'{((sorted_diff > 0).sum() / len(sorted_diff))*100:.2f}% of catchments have positive improvement')




